# Introduction to xarray 

#### presented by Ruth Moorman, Thursday 13th July 2023 (SURF student tutorial) 
#### adapted using ESE101 "Earth's Atmosphere" (2020) recitation by Dave Bonan, Jordan Benjamin, and Tapio Schneider

This tutorial uses output from a state-of-the-art climate model ([CESM2](http://www.cesm.ucar.edu/models/cesm2/)).

In addition to what we cover here I would also recommend this ["xarray in 45 min" tutorial](https://tutorial.xarray.dev/overview/xarray-in-45-min.html), which covers a lot of the basics and which I take material from here, I just wanted to use more visibly Earth-like data for this demonstration. This is all really just to get you started, xarray is **very** well documented and there's load of great resources on their pages so I recommend using them!

In [1]:
## PLOTTING PACKAGES
import matplotlib as mpl        # Python's default plotting package https://matplotlib.org/
import matplotlib.pyplot as plt 
from matplotlib import rc, rcParams
rc('font', family = 'serif')
rcParams["text.usetex"]=True
rc('xtick', labelsize=20) 
rc('ytick', labelsize=20) 
rc('axes', labelsize=20)
import cartopy.crs as ccrs      # Mapping  package https://scitools.org.uk/cartopy/docs/latest/

## ANALYSIS PACKAGES
import numpy as np              # Python's numerical and array calculations package https://numpy.org/
import xarray as xr             # Python package for labeled datasets and arrays

## Loading a "Dataset" (and the "DataArrays", "Coordinates", "Dimensions", and "Attributes" within it ...)

Below, for reference, is a example of what sorts of information can be included in an xarray Dataset as extracted from a [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file type. 


<p align="center">
  <img width="900" height="300" src="https://docs.xarray.dev/en/stable/_images/dataset-diagram.png">
</p>

Let's explore this file structure in our example netCDF file. 



In [2]:
ds = xr.open_dataset('CESM2_example_data.nc') # load the dataset from a netCDF file
ds                                               # examine the dataset

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 192, lon: 288, plev: 19, time: 12)
Coordinates:
  * time      (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * plev      (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
Dimensions without coordinates: bnds
Data variables:
    tas       (time, lat, lon) float32 ...
    ta        (time, plev, lat, lon) float32 ...
    lon_bnds  (lon, bnds) float64 ...
    lat_bnds  (lat, bnds) float64 ...
Attributes: (12/44)
    CDI:                    Climate Data Interface version 1.9.7.1 (http://mp...
    history:                Tue Sep 22 16:45:30 2020: cdo -ymonmean /export/d...
    source:                 CESM2 (2017): atmosphere: CAM6 (0.9x1.25 finite v...
    institution:            National Center for Atmospheric Research
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    sub_experiment_id:      none
    branch_time_in_parent:  219000.0
    branch_time_in_child:   674885.0
    branch_method:          standard
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NCAR.CESM2.h...
    CDO:                    Climate Data Operators version 1.9.7.1 (http://mp...

There's a lot of extra information in here, not just the arrays of raw data, that's kind of the whole point! 

Make sure to click on the icons to the right of each Dataset component, they will display either metadata about that component (square icon) or the data itself (cylindrical icon) if the data is in memory (sometimes xarray does "lazy loading" stuff with where only metadata, not array contents, are kept in memory...but I'm not going into that today!).

If we want to extract or point to a particular component of this Dataset, we use the convention `ds.{thing you want in ds}` or `ds["thing you want in ds"]`, for example, to extract just the `tas` variable (a DataArray structure) we can write:

In [3]:
ds["tas"]

<xarray.DataArray 'tas' (time: 12, lat: 192, lon: 288)>
[663552 values with dtype=float32]
Coordinates:
  * time     (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
Attributes: (12/19)
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    ...             ...
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    variable_id:    tas

Note that this doesn't actually show us the values within the array here (part of the lazy loading), so if you want to see what the array within this DataArray structure looks like in a more traditional visualization you can execute `ds["tas"].values()`.

In [4]:
ds["tas"].values

array([[[246.53833, 246.53831, 246.53833, ..., 246.53831, 246.53833,
         246.53833],
        [247.28275, 247.24307, 247.08826, ..., 247.31848, 247.30956,
         247.29736],
        [247.74873, 247.70953, 247.68953, ..., 247.97449, 247.92476,
         247.85715],
        ...,
        [246.30984, 246.3379 , 246.36473, ..., 246.23503, 246.25938,
         246.28297],
        [246.12901, 246.13977, 246.15085, ..., 246.09315, 246.10536,
         246.1175 ],
        [245.87308, 245.87418, 245.8752 , ..., 245.86902, 245.8705 ,
         245.87186]],

       [[236.95557, 236.95554, 236.95558, ..., 236.95554, 236.95554,
         236.95552],
        [237.63248, 237.59052, 237.43251, ..., 237.67442, 237.66481,
         237.65   ],
        [238.219  , 238.1761 , 238.1533 , ..., 238.45702, 238.40405,
         238.33199],
        ...,
        [245.66266, 245.69041, 245.71576, ..., 245.58755, 245.61224,
         245.63565],
        [245.23653, 245.24907, 245.26193, ..., 245.19313, 245.20827,
   

If you were then to re-run the previous cell you would see a preview of the data within the `tas` array since it's now loaded in memory.

## Data indexing and subseting

**The \#1 perk of xarray is that we get to use readable dimension information (e.g. latitude, longitude, pressure, time values) to index and analyse our data.**

Bascially, we don't need to use classical numerical indexing when working with xarray. Instead of, for example, figuring out that the equator is associated with the 20th index along the 0th axis, we can just ask for data at `lat=0`.

You can find a lot more detail on ways to index [here](https://docs.xarray.dev/en/stable/user-guide/indexing.html), but here's a couple of examples using the `.sel()` terminology (with a cheeky preview of `.interp()`). 



Say that we wanted to pull out surface air temperature (`tas`, here) from the equator, i.e. we want data where `lat` is equal to 0.

Now, if we look at the latitude coordinate of our data (e.g. run `ds["lat"]`) we should notice that there is no `lat` value precisely equal to 0. So If we try to pull out a value with that latitude we will run into an error, execute `ds.tas.sel(lat=0)` if you're curious.


Instead we would need to include a `method` for inexact matches, these are the options:

• `None (default)`: only exact matches

• `pad / ffill`: propagate last valid index value forward

• `backfill / bfill`: propagate next valid index value backward

• `nearest`: use nearest valid index value

In [5]:
ds.tas.sel(lat=0,method='ffill') # gives us the latitude prior to zero (-0.4712)

<xarray.DataArray 'tas' (time: 12, lon: 288)>
array([[300.57715, 300.59232, 300.60654, ..., 300.49686, 300.53333, 300.5608 ],
       [301.00607, 301.02072, 301.02136, ..., 300.93707, 300.9656 , 300.98495],
       [301.41086, 301.42337, 301.42447, ..., 301.3351 , 301.36526, 301.39017],
       ...,
       [299.1248 , 299.19684, 299.2948 , ..., 298.93982, 298.99918, 299.0596 ],
       [299.8663 , 299.93353, 300.0086 , ..., 299.69608, 299.74576, 299.7983 ],
       [300.24646, 300.27283, 300.3009 , ..., 300.16483, 300.19388, 300.21838]],
      dtype=float32)
Coordinates:
  * time     (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lat      float64 -0.4712
Attributes: (12/19)
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    ...             ...
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    variable_id:    tas

In [6]:
ds.tas.sel(lat=0,method='bfill') # gives us the latitude after zero (0.4712)

<xarray.DataArray 'tas' (time: 12, lon: 288)>
array([[300.76273, 300.7826 , 300.79843, ..., 300.6431 , 300.68863, 300.72513],
       [301.13058, 301.15424, 301.16364, ..., 301.01932, 301.06366, 301.1    ],
       [301.47397, 301.48544, 301.48355, ..., 301.39456, 301.4264 , 301.45087],
       ...,
       [299.3273 , 299.38492, 299.474  , ..., 299.1795 , 299.22824, 299.27704],
       [300.02307, 300.06702, 300.12915, ..., 299.90198, 299.94193, 299.9799 ],
       [300.3659 , 300.39108, 300.41968, ..., 300.29724, 300.32602, 300.34738]],
      dtype=float32)
Coordinates:
  * time     (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lat      float64 0.4712
Attributes: (12/19)
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    ...             ...
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    variable_id:    tas

Or we could start getting fancy and interpolate our data to the equator using the [`.interp()` function](https://docs.xarray.dev/en/stable/user-guide/interpolation.html).

In [7]:
ds.tas.interp(lat=0, method='linear')

<xarray.DataArray 'tas' (time: 12, lon: 288)>
array([[300.66993713, 300.68745422, 300.70248413, ..., 300.56997681,
        300.61097717, 300.64295959],
       [301.06832886, 301.08747864, 301.09249878, ..., 300.97819519,
        301.01463318, 301.04248047],
       [301.44241333, 301.45440674, 301.45401001, ..., 301.36483765,
        301.39582825, 301.42051697],
       ...,
       [299.2260437 , 299.2908783 , 299.38439941, ..., 299.05966187,
        299.1137085 , 299.1683197 ],
       [299.94468689, 300.00027466, 300.06887817, ..., 299.79902649,
        299.84384155, 299.88909912],
       [300.30618286, 300.33195496, 300.36029053, ..., 300.23103333,
        300.25994873, 300.28288269]])
Coordinates:
  * time     (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lat      int64 0
Attributes: (12/19)
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    ...             ...
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    variable_id:    tas

Another very common action would be to index according to a range, e.g. we may only want Southern Hemisphere data (sensibly, as it is the hemisphere with Australia in it).

In [8]:
ds.tas.sel(lat=slice(None,0)) # gives data with latitude values between the lower limit (None, could equivalently say -90) and the equator (0)

<xarray.DataArray 'tas' (time: 12, lat: 96, lon: 288)>
array([[[246.53833, 246.53831, ..., 246.53833, 246.53833],
        [247.28275, 247.24307, ..., 247.30956, 247.29736],
        ...,
        [300.4824 , 300.49835, ..., 300.4463 , 300.4655 ],
        [300.57715, 300.59232, ..., 300.53333, 300.5608 ]],

       [[236.95557, 236.95554, ..., 236.95554, 236.95552],
        [237.63248, 237.59052, ..., 237.66481, 237.65   ],
        ...,
        [300.9436 , 300.94702, ..., 300.91852, 300.932  ],
        [301.00607, 301.02072, ..., 300.9656 , 300.98495]],

       ...,

       [[234.73273, 234.73273, ..., 234.73273, 234.73273],
        [235.42912, 235.39182, ..., 235.45154, 235.44125],
        ...,
        [299.7156 , 299.78543, ..., 299.59048, 299.64688],
        [299.8663 , 299.93353, ..., 299.74576, 299.7983 ]],

       [[245.43279, 245.43279, ..., 245.43279, 245.43279],
        [246.12546, 246.0862 , ..., 246.15402, 246.14046],
        ...,
        [300.20035, 300.23154, ..., 300.13962, 300.16846],
        [300.24646, 300.27283, ..., 300.19388, 300.21838]]], dtype=float32)
Coordinates:
  * time     (time) object 2010-01-15 12:00:00 ... 2010-12-15 12:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... -2.356 -1.414 -0.4712
Attributes: (12/19)
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    ...             ...
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    variable_id:    tas